###  Set up Environment

In [1]:
import json
from lxml import html
from itertools import zip_longest
import pandas as pd
import time
import random
import re
import pytz
from datetime import datetime, date

### Install chromedriver

In [ ]:
# This is a special way only for Deepnote.
# Solution from https://community.deepnote.com/c/ask-anything/webdriverexception-error 
!sudo apt-get update
!sudo apt-get install chromium-driver -y

### Update datetime

In [ ]:
# Today
today = date.today()
print("today: ",today)

# Date + time
update_time = datetime.fromtimestamp(time.time())
print("current_time:",update_time)

# Start_time
start=time.perf_counter()

### Upload address.csv

In [ ]:
df = pd.read_csv('doc/address_20240329.csv')
print(df)

city = df['city'].tolist()
town = df['town'].tolist()
address = df['address'].tolist()

### Set up chromedriver

In [ ]:
!pip install selenium==4.4.0
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Set headless and no sandbox. The no sandbox is for Deepnote.
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--no-gpu')
options.add_argument('--disable-extensions')
options.add_argument('--dns-prefetch-disable')

### Scrape all swimlane data & nearby vendor data

In [6]:
col_html_content =[]
col_html_address =[]
col_phone =[]
page_status =[]
page_count =[]
page_address = []
page_time = []

In [ ]:
for i in address[0:11]:
    try:
        # 判断要使用的 cookie
        cookie_file = 'cookie/mkt_user1_cookie.json'
        phone = "xxxxx"

        # 打開對應的 cookie
        with open(cookie_file) as f:
            cookies = json.load(f)

        # 打開網頁，並加入 cookie
        if __name__ == '__main__':
            driver = webdriver.Chrome(options=options)
            driver.get('https://www.ubereats.com/tw')

            for cookie in cookies:
                driver.add_cookie(cookie)
            driver.refresh()
            
        driver.maximize_window()

        #  --- 切換地址 ---
        #找到輸入地址的地方
        driver.find_element(By.XPATH,'//*[@id="wrapper"]/header/div/div/div/div/div/div[3]/div[1]/div[2]/a/div[1]').click() 
        time.sleep(2)
        #進入變更區域。
        driver.find_element(By.XPATH,'//*[@id="wrapper"]/div[3]/div/div/div[2]/div[3]/div/div[2]/div[1]/div[2]/a').click()
        time.sleep(2)
        #選取變更區域。
        getblock1 = driver.find_element(By.XPATH,'//*[@id="location-typeahead-location-manager-input"]')
        #輸入要變更的地址。
        getblock1.send_keys(i) 
        time.sleep(3)
        #按下Enter送出地址
        getblock1.send_keys('\ue007')
        time.sleep(3)
        print(i, "換地址成功","當前地址：",driver.find_element(By.XPATH,'//*[@id="wrapper"]/header/div/div/div/div/div/div[3]/div[1]/div[2]/a/div[1]').text)
        
        #  --- 展開更多店家 

        for click in range(1,12):
            try:
                button = driver.find_element(By.XPATH, '//*[@id="main-content"]/div/div[5]/div/div/button')
                button_text = button.text
                button.click()
                time.sleep(5)
                page_status.append(button_text)
                page_count.append(click)
                page_address.append(i)
                page_time.append(datetime.fromtimestamp(time.time()))
            except:
                pass
        print(i,"更多店家展開完成")
        time.sleep(3)
        
        #  --- 開始爬取整頁資料 ---

        html_content = driver.page_source
        col_html_content.append(html_content)
        col_html_address.append(i)
        col_phone.append(phone)

        print(i,"資料爬取完成")    

       # --- 單一地址資料抓取結束 ---  

       # --- 頁面結束畫面screenshot
        driver.set_window_size(1200, 1000)
        picture_name = "screenshot/" + "end_" + i + "_" + str(update_time) + ".png"
        driver.save_screenshot(picture_name)
       
       # --- 關閉網站
        driver.close() 

    except:
        driver.close() 
       
print("地址查詢完成")

### Expand more restaurants dataframe

In [ ]:
page_data = {
    'page_address': page_address,
    'page_status': page_status,
    'page_click': page_count,
    'page_time': page_time,
}

page_df = pd.DataFrame(page_data)
print(page_df)

### Get HTML data

In [ ]:
scrape_time = [update_time] * len (col_html_content)
html_data= pd.DataFrame({'html_address':col_html_address,'html_content':col_html_content, 'phone':col_phone, 'scrape_time_utc': scrape_time})
html_data

### Clean html data

In [10]:
from lxml import html
from itertools import zip_longest

In [11]:
col_swimlane = []
col_address = []
col_status = []
col_vendor = []
col_delivery_fee = []
col_delivery_time = []
col_comment = []
col_user = []

### UE Swimlane

In [ ]:
# switch html
for a in range(0,11):
    try:
        tree = html.fromstring(col_html_content[a])
    # switch swimlane
        for k in range(1,40):
            try:
                xpath_swimlane = f'//*[@id="main-content"]/div/div[5]/div/div/div[2]/div[{k}]/div/section/div[1]/div[1]/div/div[1]/a/div'
                swimlane = tree.xpath(xpath_swimlane)
                for m in range(1,5):
                    try:
                        for l in range(1,5):
                            try:
                                xpath_vendor_name = f'//*[@id="main-content"]/div/div[5]/div/div/div[2]/div[{k}]/div/section/div[2]/ul[{m}]/li[{l}]/div/div/div/div[2]/div[1]/div[1]/div'
                                xpath_delivery_fee = f'//*[@id="main-content"]/div/div[5]/div/div/div[2]/div[{k}]/div/section/div[2]/ul[{m}]/li[{l}]/div/div/div/div[2]/div[2]/div[1]/span/div/span/span'
                                xpath_delivery_time = f'//*[@id="main-content"]/div/div[5]/div/div/div[2]/div[{k}]/div/section/div[2]/ul[{m}]/li[{l}]/div/div/div/div[2]/div[2]/div[2]/span[2]/div'
                                xpath_comment = f'//*[@id="main-content"]/div/div[5]/div/div/div[2]/div[{k}]/div/section/div[2]/ul[{m}]/li[{l}]/div/div/div/div[2]/div[1]/span'
                                vendor_name = tree.xpath(xpath_vendor_name)
                                delivery_fee = tree.xpath(xpath_delivery_fee)
                                delivery_time = tree.xpath(xpath_delivery_time)
                                comment = tree.xpath(xpath_comment)
                                for d,e,f,g,h in zip_longest(swimlane,vendor_name, delivery_fee, delivery_time, comment, fillvalue=None):
                                    if d is not None and e is not None and f is not None and g is not None and h is not None:
                                        col_swimlane.append(d.text_content())
                                        col_address.append(col_html_address[a])
                                        col_vendor.append(e.text_content())
                                        col_delivery_fee.append(f.text_content())
                                        col_delivery_time.append(g.text_content())
                                        col_comment.append(h.text_content())
                                        col_status.append('NA')
                                        col_user.append(col_phone[a])
                                    else:
                                        pass
                            except:
                                pass
                    except:
                        pass
            except: 
                pass
    except:
        pass
print("Complete Swimlane Scraping!")

In [ ]:
# count of swimlane data
print("address", len(col_address),
      "swimlane", len(col_swimlane),
      "vendor_name", len(col_vendor),
      "delivery_fee",len(col_delivery_fee),
      "delivery_time",len(col_delivery_time),
      "comment",len(col_comment),
      "status",len(col_status),
      "user",len(col_user))

### UE nearby vendors 

In [ ]:
for a in range(0,11):
    try:
        tree = html.fromstring(col_html_content[a])
        for m in range(0,700):
            try: 
                xpath_status = f'//*[@id="main-content"]/div/div[5]/div/div/div[2]/div[{m}]/div/div/div/div/div[1]/figcaption/div[2]/div/div'
                status = tree.xpath(xpath_status)
                xpath_vendor_name = f'//*[@id="main-content"]/div/div[5]/div/div/div[2]/div[{m}]/div/div/div/a/h3'
                vendor_name = tree.xpath(xpath_vendor_name)
                xpath_delivery_fee = f'//*[@id="main-content"]/div/div[5]/div/div/div[2]/div[{m}]/div/div/div/div/div[2]/div[2]/div[1]/span/div/span/span'
                delivery_fee = tree.xpath(xpath_delivery_fee)
                xpath_delivery_time = f'//*[@id="main-content"]/div/div[5]/div/div/div[2]/div[{m}]/div/div/div/div/div[2]/div[2]/div[2]/span[2]/div'
                delivery_time = tree.xpath(xpath_delivery_time)
                xpath_comment = f'//*[@id="main-content"]/div/div[5]/div/div/div[2]/div[{m}]/div/div/div/div/div[2]/div[1]/span'
                comment = tree.xpath(xpath_comment)
                for o, p, q, r, s in zip_longest(status, vendor_name, delivery_fee, delivery_time, comment, fillvalue='NA'):
                    col_swimlane.append("附近店家")
                    col_address.append(col_html_address[a])
                    col_status.append(o.text_content() if o != 'NA' else 'NA')
                    col_vendor.append(p.text_content() if p != 'NA' else 'NA')
                    col_delivery_fee.append(q.text_content() if q != 'NA' else 'NA')
                    col_delivery_time.append(r.text_content() if r != 'NA' else 'NA')
                    col_comment.append(s.text_content() if s != 'NA' else 'NA') 
                    col_user.append(col_phone[a])
            except:
                pass
    except:
        pass
print("Complete Nearby Vendors Scraping!")

In [ ]:
# count of swimlane + nearby vendors data
print("address", len(col_address),
      "swimlane", len(col_swimlane),
      "vendor_name", len(col_vendor),
      "delivery_fee",len(col_delivery_fee),
      "delivery_time",len(col_delivery_time),
      "comment",len(col_comment),
      "status",len(col_status),
      "user",len(col_user))

In [16]:
# Create DataFrame
ue_data = pd.DataFrame({'swimlane':col_swimlane,
                        'address': col_address,
                        'status':col_status, 
                        'vendor':col_vendor, 
                        'df':col_delivery_fee, 
                        'dt':col_delivery_time,
                        'comment':col_comment})

ue_data['orderable'] = ue_data['status'] == 'NA'
col_orderable = ue_data['orderable']
ue_data

,swimlane,address,status,vendor,df,dt,comment,orderable
0,日用品外送到府,400台中市中區臺灣大道一段1號,NA,統一超商 綠川門市,5.00TWD 費用,10–30 分鐘,在 5 星評分中獲得 4.6 顆星的高人氣餐廳 - 根據 99 則顧客評論。,True
1,日用品外送到府,400台中市中區臺灣大道一段1號,NA,日藥本舖 台中一中門市,5.00TWD 費用,10–25 分鐘,在 5 星評分中獲得 5.0 顆星的高人氣餐廳 - 根據 58 則顧客評論。,True
2,日用品外送到府,400台中市中區臺灣大道一段1號,NA,萊爾富4723台中夜市店,5.00TWD 費用,10–30 分鐘,在 5 星評分中獲得 4.8 顆星的高人氣餐廳 - 根據 200 則以上的顧客評論。,True
3,日用品外送到府,400台中市中區臺灣大道一段1號,NA,全家Famisuper 台中金福仁店_023225,5.00TWD 費用,10–25 分鐘,在 5 星評分中獲得 4.8 顆星的高人氣餐廳 - 根據 200 則以上的顧客評論。,True
4,日用品外送到府,400台中市中區臺灣大道一段1號,NA,Uber Eats 優市 精武店,5.00TWD 費用,10–25 分鐘,在 5 星評分中獲得 4.9 顆星的高人氣餐廳 - 根據 200 則以上的顧客評論。,True
...,...,...,...,...,...,...,...,...
5583,附近店家,701台南市東區中華東路一段366號,NA,蒸滿足 成大店,NA,NA,在 5 星評分中獲得 4.9 顆星的高人氣餐廳 - 根據 200 則以上的顧客評論。,True
5584,附近店家,701台南市東區中華東路一段366號,NA,小得飲滿 民族店,NA,NA,在 5 星評分中獲得 4.9 顆星的高人氣餐廳 - 根據 94 則顧客評論。,True
5585,附近店家,701台南市東區中華東路一段366號,NA,京享鐵板燒 忠義店,NA,NA,在 5 星評分中獲得 4.8 顆星的高人氣餐廳 - 根據 200 則以上的顧客評論。,True
5586,附近店家,701台南市東區中華東路一段366號,NA,廣西老友麵館 開元店,NA,NA,在 5 星評分中獲得 4.7 顆星的高人氣餐廳 - 根據 200 則以上的顧客評論。,True


### Organize final dataframe

In [17]:
update_time_utc = [update_time] * len (col_delivery_fee)
date = [today] * len (col_delivery_fee)

In [18]:
data= pd.DataFrame({'date_utc':date,
                    'update_time_utc':update_time_utc,
                    'address':col_address,
                    'swimlane':col_swimlane, 
                    'vendor_name':col_vendor, 
                    'delivery_fee':col_delivery_fee,
                    'delivery_time':col_delivery_time,
                    'comment':col_comment,
                    'user':col_user,
                    'is_orderable':col_orderable,
                    'status': col_status
                    })

In [19]:
# combine address related data
result = pd.merge(data, df, on ='address', how = 'left')
result['date_utc'] = pd.to_datetime(result['date_utc'])

# remove closing vendors
filtered_df = result[result['status'] != '已關閉']
filtered_df = result[result['vendor_name'] != 'NA']
result = filtered_df.copy()

In [20]:
# Extracting values using lambda and apply functions
result = result.assign(
    df_value=result["delivery_fee"].apply(lambda x: float(x.split("TWD 費用")[0]) if isinstance(x, str) and "TWD 費用" in x else 0),
    dt_value=result["delivery_time"].apply(lambda x: int(x.split("–")[1].split()[0]) if isinstance(x, str) and "–" in x else 0),
    comment_score=result["comment"].apply(lambda x: float(x.split('獲得')[1].split('顆星')[0]) if isinstance(x, str) and '獲得' in x and '顆星' in x else 0),
    comment_count=pd.to_numeric(result["comment"].str.split("根據 ").str[1].str.split(" ").str[0], errors='coerce').fillna(0).astype('Int64')
)

# add col_rank
result['rank'] = result.groupby("address").cumcount() + 1

In [ ]:
# Reordering columns
result = result[['rank', 'user','date_utc', 'update_time_utc', 'city', 'town', 
                 'address', 'swimlane', 'vendor_name', 'delivery_fee', 'df_value', 
                 'delivery_time', 'dt_value', 'comment', 'comment_score', 'comment_count','is_orderable']]


result

### Upload dataframe to GCP

pandas to gcp handbook: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_gbq.html

In [ ]:
!pip install pandas_gbq==0.20.0

In [24]:
import pandas as pd
from datetime import datetime
import re
import google.oauth2.credentials
import pandas_gbq
import json

In [ ]:
f = open('gcp_info/user_account.json') # add GCP token
credential = json.load(f)

credentials = google.oauth2.credentials.Credentials(
    credential['token'],
    refresh_token=credential['refresh_token'],
    token_uri=credential['token_uri'],
    client_id=credential['client_id'],
    client_secret=credential['client_secret']
    )

pandas_gbq.context.credentials = credentials

In [ ]:
project_id = 'database_name'
table_id = 'table_name'

result.to_gbq(destination_table = table_id, 
              project_id = project_id, 
              credentials = credentials, 
              if_exists='append')

### Complete UE Scraper

In [29]:
end=time.perf_counter()

In [ ]:
print("執行時間: %f 秒" % (end-start))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ca7b9e5b-3059-42bb-8cc1-8f6e198045f7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>